# Automatically Download AU Stock List and Info

## Import

Libraries

In [6]:
import requests
import os
import pandas as pd
from tqdm import tqdm
import time
import random

Definitions

In [2]:
# current dir + data
download_dir = os.path.join(os.getcwd(), '../data')

## Get ASX List

In [3]:
# todo: does this work tmr
# The URL you provided
url = "https://asx.api.markitdigital.com/asx-research/1.0/companies/directory/file?access_token=83ff96335c2d45a094gics_df02a206a39ff4"

# Download the file
response = requests.get(url)

# Save the file if the request is successful
if response.status_code == 200:
    with open(f"{download_dir}/asx_companies_directory.csv", "wb") as file:
        file.write(response.content)
    print("File downloaded successfully!")
else:
    print(f"Failed to download file. Status code: {response.status_code}")

File downloaded successfully!


## Get gics sector and industry

In [4]:
asx_companies_directory = pd.read_csv(
    f"{download_dir}/asx_companies_directory.csv")

ASX_codes = asx_companies_directory['ASX code'].to_list()

In [7]:
import yfinance as yf

# Create a dataframe to store stock information
stock_data = []

for ticker in tqdm(ASX_codes):
    ticker += ".AX"
    stock = yf.Ticker(ticker)
    info = stock.info
    stock_data.append({
        'Ticker': ticker,
        'Name': info.get('shortName', 'N/A'),
        'Market Cap': info.get('marketCap', 'N/A'),
        'Sector': info.get('sector', 'N/A'),
        'Industry': info.get('industry', 'N/A'),
    })
    time.sleep(random.uniform(0.05, 0.1))

# Convert the data into a DataFrame
gics_df = pd.DataFrame(stock_data)

# Display the DataFrame
display(gics_df)

100%|██████████| 1930/1930 [23:02<00:00,  1.40it/s]


,Ticker,Name,Market Cap,Sector,Industry
0,14D.AX,1414DEGREE FPO [14D],6497523,Utilities,Utilities - Independent Power Producers
1,1AD.AX,ADALTA FPO [1AD],9471870,Healthcare,Biotechnology
2,1AE.AX,AURORAENER FPO [1AE],12713544,Energy,Uranium
3,1AI.AX,ALGORAE FPO [1AI],11811730,Healthcare,Biotechnology
4,1CG.AX,1CLICKLIFE FPO [1CG],11778800,Technology,Software - Application
...,...,...,...,...,...
1925,ZLD.AX,ZELIRATHER FPO [ZLD],7148736,Healthcare,Biotechnology
1926,ZMI.AX,ZINCOFIRE FPO [ZMI],6237121,Basic Materials,Other Industrial Metals & Mining
1927,ZMM.AX,ZIMILMITED FPO [ZMM],5417216,Industrials,Electrical Equipment & Parts
1928,ZNC.AX,ZENITH MIN FPO [ZNC],14667875,Basic Materials,Other Industrial Metals & Mining


In [8]:
gics_df.to_csv('../data/asx_gics.csv', index=False)